**Table of contents**<a id='toc0_'></a>

-   1. [Using Sympy](#toc1_)
    -   1.1. [Propositional Logic](#toc1_1_)
    -   1.2. [Predicate Logic](#toc1_2_)
-   2. [Using z3 solver](#toc2_)

<!-- vscode-jupyter-toc-config
	numbering=true
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->


In [67]:
import sympy as sp
from sympy import symbols
from sympy.logic.boolalg import And, Implies, Not
from sympy.logic import inference

sp.__version__

'1.13.3'

# 1. <a id='toc1_'></a>[Using Sympy](#toc0_)


-   Major premise: All humans are mortal.
-   Minor premise: All Greeks are humans.
-   Conclusion/Consequent: All Greeks are mortal.


In [248]:
propositions = [
    "G & ~Ev",
    "~Ev",  # Evil exists.
    "~G >> R",
    "~C & D >> G",
    "G >> ~R",
]

conclusion = "R"  # Testing if this leads to an inconsistency (contradiction).

## 1.1. <a id='toc1_1_'></a>[Propositional Logic](#toc0_)


In [249]:
class PropositionalLogicSyllogism:
    global_statement = None
    simplified_statement = None

    def __init__(self, propositions):
        self.propositions = propositions
        self.global_statement = And(*sp.sympify(self.propositions))
        self.simplified_statement = sp.simplify_logic(
            self.global_statement, force=False
        )
        # display(self.global_statement, "<=>", self.simplified_statement)
        # print("---"*50)
        # display(list(inference.satisfiable(self.simplified_statement, all_models=True)))

    def representation(self, simplify=False):
        if simplify:
            return sp.simplify_logic(self.global_statement, force=False)
        else:
            return self.global_statement

    @property
    def symbols(self):
        return self.global_statement.atoms() | self.global_statement.free_symbols

    def solve(self, negate=False):
        statement = (
            Not(self.simplified_statement)
            if negate
            else self.simplified_statement.copy()
        )
        # check if all possible assignments are valid
        if inference.valid(statement):
            return "Tautology: Valid in all Possible Worlds"
        else:
            models = list(
                inference.satisfiable(statement, algorithm=None, all_models=True)
            )
            if not any(models):
                return "Invalid in all Possible Worlds: contraduition"
            else:
                n_worlds = f"{len(models)} world{'s' if len(models) > 1 else ''}"
                # print(f"Valid in {n_worlds}: satifiable")
                return [model for model in models]

    @staticmethod
    def check_entailment(premises, conclusion) -> bool | str:
        # print(sp.sympify(premises, evaluate=True))
        know_base: inference.PropKB = inference.PropKB()
        [know_base.tell(premise) for premise in sp.sympify(premises)]
        print(f"{premises} ⊨ {conclusion} is:")
        if not sp.simplify_logic(And(*sp.sympify(premises))):
            return "Invalid Premises: Contradiction"
        return know_base.ask(conclusion)

In [253]:
PropositionalLogicSyllogism.check_entailment(
    premises=propositions,
    conclusion=conclusion,
)

['G & ~Ev', '~Ev', '~G >> R', '~C & D >> G', 'G >> ~R'] ⊨ R is:


False

In [254]:
problem = PropositionalLogicSyllogism(propositions)
problem.symbols
display(problem.representation(simplify=True))
valid_worlds: list[dict] = problem.solve(negate=False)
valid_worlds


G & ~C & ~Ev & ~R

[{G: True, C: False, R: False, Ev: False}]

In [256]:
def print_worlds(valid_worlds:dict|str) -> None:
    if isinstance(valid_worlds, str):
        print(valid_worlds)
        return
    print("-------logic is satisfiable in some possible worlds--------")
    for k in sorted(valid_worlds[0].keys(), key=lambda x: str(x)):
        print(f"{str(k):^10}", end="")
    print("\n",f"{'-'*9*len(valid_worlds[0])}")
    for world in valid_worlds:
        world = dict(sorted(world.items(), key=lambda x: str(x)))
        for value in world.values():
            print(f"{value:^10}", end="")
        print()

print_worlds(valid_worlds)

-------logic is satisfiable in some possible worlds--------
    C         Ev        G         R     
 ------------------------------------
    0         0         1         0     


## 1.2. <a id='toc1_2_'></a>[Predicate Logic](#toc0_)


In [56]:
from sympy.logic.boolalg import Implies
from sympy import symbols, Symbol
from sympy.logic.inference import satisfiable


# Define the Predicate class
class Predicate:
    def __init__(self, name):
        self.name = name

    def __call__(self, *args):
        args_str = ",".join(map(str, args))
        return Symbol(f"{self.name}({args_str})")


# Define our predicates
P = Predicate("P")  # Predicate P(x)
H = Predicate("H")  # Predicate H(y)
B = Predicate("B")  # Predicate B(x, y)

# Define variables
x1 = Symbol("x1")
y1 = Symbol("y1")

# Build the formula: ∀x. P(x) → (∃y. H(y) ∧ B(x, y))
# We'll demonstrate this for specific instances x1 and y1
formula = Implies(P(x1), H(y1) & B(x1, y1))

# Print the formula
print("Logical Formula:")
print(formula)

# Check if the formula is satisfiable
result = satisfiable(formula)
print("\nIs the formula satisfiable?")
print(result is not False)

if result:
    print("\nOne possible satisfying assignment:")
    for key, value in result.items():
        print(f"{key}: {value}")


Logical Formula:
Implies(P(x1), B(x1,y1) & H(y1))

Is the formula satisfiable?
True

One possible satisfying assignment:
H(y1): True
B(x1,y1): True
P(x1): False


# 2. <a id='toc2_'></a>[Using z3 solver](#toc0_)


In [317]:
from z3 import Bools, Solver, And, Bool
problem.symbols
s = Solver()
sym_mapto_z3 = {str(x):Bool(str(x)) for x in problem.symbols}
s.add(sym_mapto_z3['G'])
s.add(sym_mapto_z3['R'] | sym_mapto_z3['G'])
s.check()

sat

In [ ]:
def propositional_logic_syllogism_z3():
    from z3 import Bool, And, Implies, Not, Solver, unsat, prove, extract_model

    # Define atomic propositions
    P = Bool("P")  # P: All humans are mortal.
    Q = Bool("Q")  # Q: All Greeks are humans.
    R = Bool("R") 
    R = "P & Q"
    print(R)
    R = Bool("R")  # R: All Greeks are mortal.
    # Create a solver
    s = Solver()

    # Add premises (assume P and Q are true)
    s.add(P)
    s.add(Q)

    # Attempt to prove that P and Q imply R
    # In propositional logic, without additional implications, we cannot derive R from P and Q
    # Let's check if (P ∧ Q ∧ ¬R) is satisfiable
    s.push()
    s.add(Not(R))

    print("Propositional Logic Syllogism with Z3:")
    
    print("--------------------------------------")
    if s.check() == unsat:
        print("The conclusion (R) follows from the premises.")
    else:
        print(
            "The conclusion does NOT necessarily follow from the premises in propositional logic."
        )
        print("Counterexample Model:")
        print(s.model())


def predicate_logic_syllogism_z3():
    from z3 import (
        Solver,
        ForAll,
        Implies,
        Function,
        BoolSort,
        Const,
        Not,
        And,
        unsat,
        prove,
        DeclareSort,
    )

    # Declare a sort for Person
    Person = DeclareSort("Person")

    # Define variables
    x = Const("x", Person)

    # Define predicates
    H = Function("H", Person, BoolSort())  # H(x): x is human
    M = Function("M", Person, BoolSort())  # M(x): x is mortal
    G = Function("G", Person, BoolSort())  # G(x): x is Greek

    # Major premise: ∀x (H(x) ⇒ M(x))
    major_premise = ForAll([x], Implies(H(x), M(x)))

    # Minor premise: ∀x (G(x) ⇒ H(x))
    minor_premise = ForAll([x], Implies(G(x), H(x)))

    # Conclusion: ∀x (G(x) ⇒ M(x))
    conclusion = ForAll([x], Implies(G(x), M(x)))

    # Attempt to prove that the conclusion follows from the premises
    print("Predicate Logic Syllogism with Z3:")
    print("------------------------------------")
    print("Attempting to prove that the conclusion follows from the premises...")

    # Use Z3's prove function
    prove(Implies(And(major_premise, minor_premise), conclusion))


# Run the functions
propositional_logic_syllogism_z3()
print("\n")
predicate_logic_syllogism_z3()

P & Q
Propositional Logic Syllogism with Z3:
--------------------------------------
The conclusion does NOT necessarily follow from the premises in propositional logic.
Counterexample Model:
[R = False, Q = True, P = True]


Predicate Logic Syllogism with Z3:
------------------------------------
Attempting to prove that the conclusion follows from the premises...
proved
